<a href="https://colab.research.google.com/github/adey4/food-sales-predictions/blob/main/Project_1_Part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 - Part 5
Ankit Dey

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(display='diagram')

In [17]:
sales_predictions = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSwXLcK1gHn4S6xdVnTN1kZIAdiR3EGWqJKzxOoqEKOdk--tVrbdN_aIQN-PgCVUSyMU_R7KSHsGkOV/pub?gid=135014557&single=true&output=csv'
df = pd.read_csv(sales_predictions)

In [18]:
# Explore dataset
display(df.info())
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


None

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [19]:
# Check for duplicates
df.duplicated().sum()

0

In [20]:
# Identify missing values
display(df.isna().sum())
print('\n')
print(f"Item_Weight column is missing {round((df['Item_Weight'].isna().sum() / len(df['Item_Weight']) * 100), 1)}% of its rows.")
print(f"Outlet_Size column is missing {round((df['Outlet_Size'].isna().sum() / len(df['Outlet_Size']) * 100), 1)}% of its rows.")
print(f"Item_Weight column contains {len(df['Item_Weight'].dropna())} non-missing values.")
print(f"Outlet_Size column contains {len(df['Outlet_Size'].dropna())} non-missing values.")

# Item_Weight and Outlet_Size columns are missing a moderate percentage of data, but still contain enough data points for column statistics to be useful
# Item_Weight and Outlet_Size may be correlated to sales -> columns should not be dropped
# Values will be imputed after train-test split to prevent data leakage in our model

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64



Item_Weight column is missing 17.2% of its rows.
Outlet_Size column is missing 28.3% of its rows.
Item_Weight column contains 7060 non-missing values.
Outlet_Size column contains 6113 non-missing values.


In [21]:
# Identify syntax errors in categorical data
data_types = df.dtypes
str_cols = data_types[data_types == 'object'].index

for col in str_cols:
  print(f'- {col}:')
  print(df[col].value_counts(dropna=False))
  print('\n\n')

- Item_Identifier:
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64



- Item_Fat_Content:
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64



- Item_Type:
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64



- Outlet_Identifier:
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    9

In [22]:
# Fix syntax errors in Item_Fat_Content column
df['Item_Fat_Content'] = df['Item_Fat_Content'].str.replace('LF', 'Low Fat')
df['Item_Fat_Content'] = df['Item_Fat_Content'].str.replace('low fat', 'Low Fat')
df['Item_Fat_Content'] = df['Item_Fat_Content'].str.replace('reg', 'Regular')
df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [23]:
# Identify errors in numerical data
df.describe() # No errors

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [24]:
# Define features matrix and target vector
X = df.drop(columns=['Item_Outlet_Sales'])
y = df['Item_Outlet_Sales']

# Split data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [25]:
# Instantiate transformers
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore')

mean_imputer = SimpleImputer(strategy='mean')
constant_imputer = SimpleImputer(strategy='constant', fill_value='missing')
most_frequent_imputer = SimpleImputer(strategy='most_frequent')

In [26]:
# Instantiate ordinal encoder
item_fat_content_labels = ['Low Fat', 'Regular']
outlet_size_labels = ['Small', 'Medium', 'High']

ordered_labels = [item_fat_content_labels, outlet_size_labels]

ordinal_encoder = OrdinalEncoder(categories=ordered_labels)

In [27]:
# Instantiate pipelines for nominal, ordinal, and numerical data
number_pipe = make_pipeline(mean_imputer, scaler)
nominal_pipe = make_pipeline(constant_imputer, ohe)
ordinal_pipe = make_pipeline(most_frequent_imputer, ordinal_encoder)

In [28]:
# Create tuples to match transformations with columns
num_selector = make_column_selector(dtype_include='number')
ord_cols = ['Item_Fat_Content', 'Outlet_Size']
nom_cols = ['Item_Identifier', 'Item_Type', 'Outlet_Identifier', 'Outlet_Location_Type', 'Outlet_Type']

num_tuple = (number_pipe, num_selector)
ord_tuple = (ordinal_pipe, ord_cols)
nom_tuple = (nominal_pipe, nom_cols)

In [29]:
# Instantiate column transformer
preprocessor = make_column_transformer(ord_tuple,
                                       nom_tuple,
                                       num_tuple,
                                       remainder='drop')

In [30]:
# Fit preprocessor then transform train and test data
preprocessor.fit(X_train)

X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)